<a href="https://colab.research.google.com/github/Brycexxx/fastai/blob/master/lesson2_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lesson2

### 1. 相关术语定义

- Learning rate：学习率，乘以梯度，决定了一次性更新权重多少
- Epoch：一个 epoch 完整遍历整个数据集，当 epoch 取得过大时，由于模型“见过”数据集太多次，容易造成过拟合
- Mini-batch：整个数据集中随机的一小部分数据集，用于数据集过大的时候能够更快的更新权重参数
- Stochastic  gradient descent：sgd，随机梯度下降，利用 mini-batches 更新权重
- Model / Architecture：模型/结构，在某种意义上来说是一样的，简单来说就是你正在更新的权重参数对应的数学函数，比如 $y = ax$
- Parameters：参数，或者说系数，更常用的说法是**weights**，也就是你正在更新的数字
- Loss function：告诉我们离正确结果有多远

### 2. 如何创建一个好的验证集

验证集选取的好坏通常决定了开发环境模型结果与生产环境的模型结果的差异。由于数据固有的性质，选取验证集可能是最重要的一步。虽然  [sklearn](https://scikit-learn.org/stable/index.html) 提供了 `train_test_split` 方法，但这对于真实世界的问题通常不是一个好的选择。

训练集、验证集和测试集的定义在有些情况下是有细微差别的。在深度学习社区，测试时间的推断通常指评估生产中的数据，这和测试集的技术定义不一样。再如上文所说 [sklearn](https://scikit-learn.org/stable/index.html) 提供的是 `train_test_split` 而不是 `train_validation_test_split` 。[kaggle](https://www.kaggle.com/) 只提供了训练集和测试集，通常为了更好的效果，需要将分出训练集的一部分作为验证集。同时，[kaggle](https://www.kaggle.com/) 提供的测试集也会被分为两个子集。

##### 1）首先，什么是验证集？

- 训练集用作训练一个模型
- 验证集用于选择模型（比如，对于一个具体的问题，是随机森林的效果好还是神经网络的效果好？或者说一个随机森林是由40个树组成好还是50个树组成更好？）
- 测试集用于检验通过验证集选出的模型效果到底如何。如果你尝试了很多不同的模型，最后可能选出一个偶然情况下在验证集下表现良好的模型，此时就可以通过测试集检验该模型是不是只是偶然得到较好的结果

**关键就是，验证集和测试集必须代表了在将来可能遇到的新数据**

##### 2）什么时候随机子集不够好？

先看几个来自 [kaggle](https://www.kaggle.com/) 的例子，它们比较具有代表性

**时间序列**

假如有如下时间序列数据：

<img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190204212008.jpg"/>

加入采取随机选取，则训练集可能如下：

<img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190204212322.jpg"/>

如上图，训练集缺失的点和容易被填补并不满足工作中的需求，同时破坏了数据的序列性

通常将某一个时间点前的所有数据作为训练集，其后的所有数据作为验证集，此时训练集如下：

<img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190204212710.jpg"/>

 [kaggle](https://www.kaggle.com/) 上有一个比赛 [predict the sales in a chain of Ecuadorian grocery stores](https://www.kaggle.com/c/favorita-grocery-sales-forecasting) ，这个比赛给出的训练数据日期从 2013.1.1 到 2017.8.15 ，测试数据从 2017.8.16 到 2017.8.31 ，一个较好的做法是选择 2017.8.1 到 2017.8.15 之间的数据作为验证集，2017.8.1 之前的所有数据作为训练集

##### 3）新的元素

你还需要考虑你在生产环境中用到的数据和你用来训练模型的数据质量有什么不同。

比如保险公司需要分辨司机开车的状态，比如有这么三个状态：打字、吃东西、认真地看向前方；你最感兴趣的应该是模型在没见过的人上的表现。

<img src="https://raw.githubusercontent.com/Brycexxx/Images/master/20190204221847.jpg"/>

加入上面两张图片，一个在训练集，一个在验证集，那么模型将会在这个图片上表现得更好比验证集中其他没出现过的人。还有一点要注意的是，如果搜集到的图片中出现的所有人恰好全部出现在训练集中，那=那么模型很有可能过拟合，它学习到的将是这些特定的人而不是人在开车时的状态。

##### 4）交叉验证的危险

[sklearn](https://scikit-learn.org/stable/index.html) 没有 `train_validation_test_split` 的原因是它假定你经常使用 `cross_validation` ，然而交叉验证的问题是它很难适用于真实世界的问题（因为它的做法就是随机，而以上叙述了随机将会带来的问题）。 `cross_validation` 仅仅在可以通过随机打乱数据选取验证集的时候有效

##### 5）[Kaggle](https://www.kaggle.com/) 的训练集等于你的训练集加上验证集

进行 [kaggle](https://www.kaggle.com/) 的一个好处就是你想做的更好就必须更加认真地考虑验证机的划分，*kaggle* 通常只提供两个可下载的数据集——训练集和测试集，测试集是不包括待预测变量的。由于 `kaggle` 一天只允许提交两次再加上为了获得更好的效果，你必须自己将下载的训练集认真地分割，利用更小的训练集训练模型，利用分割出的验证集挑选模型，配合测试集以得到更好的模型

> 翻译自 https://www.fast.ai/2017/11/13/validation-sets/

### 3. 如何处理图像分类中输入图片不属于分类模型中任一类型的情况

主要有两种解决办法：a. 对 `softmax` 给出的后验概率设置一个阈值，b.再加入一个**其他类** 

a. 如果不属于任一类的输入和其他能够得到正确分类的输入具有同样的本质属性，例如具有正确语法的句子和非语法输入。此时设置阈值可得到不错的效果。

b. 如果不属于任一类的输入和其他能够得到正确分类的输入具有同样的本质属性且模型的类别较少，或者不属于任一类的输入和其他输入具有完全不同的属性，此时设置阈值就会失效。对于第一种情况，类别少，比如只有两类，那么此时必有一类的后验概率大于 0.5，阈值设置为 0.5 就失效了；当不属于任一类的输入和其他输入具有完全不同的属性时，各类的后验概率则不会稳定的小于 0.5，此时需要设置一个新类**其他类**，以对应不属于任一类的输入。

> 参考自 https://stackoverflow.com/questions/43578715/how-best-to-deal-with-none-of-the-above-in-image-classification

### 4. 关于过拟合

很多教程都会说当你的训练集误差或损失低于验证集时，那么模型很可能过拟合了；而实际上，任何一个被正确训练的模型，其训练集误差总是低于验证集的，过拟合的表现时精度开始降低或者说损失开始升高

### 5. error_rate

等于 1 - accuracy，针对验证集

### 6. 对于学习率

通常在 `unfreeze` 之后默认为 3e-3（来自大佬的经验），`unfreeze` 之后通常传入一个 `slice=(xxx, 3e-4)`，xxx 下限通常根据 `lr_find` 确定，当 `lr_find` 的曲线在某点具有最大的斜率、且稳定，则选择此点作为下限

### 7. 怎么知道数据量够不够

当找到合适的参数后，训练足够的时间，发现在某个点精度开始下降，但这个精度并不能让人满意，此时可能就是数据量不够，找到更多的数据也许可以解决问题。这个过程没有捷径。通常你需要的数据量比你想象的要少，所以，通常用少量的数据进行尝试，以便知道下一步怎么做  